# test_lensed_quasar_time_delay_8sample

8-lens quasar time-delay forecast from `data/test_data.py`.

Key logic:
- Use only the longest time-delay pair (`max |dt|`) for each target.
- TDCOSMO fiducial cosmology: `H0=70`, `Omegam=0.3`.
- Set MST population truth to `lambda_mean=1.0`, `lambda_sigma=0.1` and draw per-lens `lambda_true`.
- External convergence is per-lens only (no population-level inference):
  - `kext_true ~ N(0, 0.01)`
  - `kext_obs = clean/noisy`
  - `kext_err = 0.01`
- Use `lambda_eff = (1-k_ext)*lambda` in both mock generation and inference likelihood.
- Use kinematic errors to define per-lens MST measurement uncertainty via `frac(lambda_err)=2*frac(sigma_v_err/sigma_v)`.
- Treat provided time delays as unbiased base values and build `t_measured_true = lambda_eff_true * t_base`.
- Each scenario runs with both `clean` and `noisy` observations.
- Three scenarios:
  1) `fiducial_no_extra`: infer lambda population and use MST measurements with no extra 8% MST error.
  2) `fiducial_plus8`: infer lambda population and add 8% (`TDCOSMO_increase_error=0.08`) to individual MST measurement error.
  3) `population_only_no_mst_measure`: no MST measurement term and fixed known lambda population (`lambda_mean`, `lambda_sigma`).
- Inference always samples `Omegam` and `H0`; first two scenarios also sample `lambda_mean` and `lambda_sigma`.



In [ ]:
import os
import sys
from pathlib import Path
import importlib.util

import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import numpyro
import numpyro.distributions as dist
from numpyro.infer import NUTS, MCMC, init_to_value
from jax import random
import arviz as az
from corner import corner

os.environ.setdefault('HDF5_USE_FILE_LOCKING', 'FALSE')

cwd = Path.cwd().resolve()
repo_root = None

if (cwd / 'slcosmo').is_dir() and (cwd / 'data').is_dir():
    repo_root = cwd
elif (cwd / 'LensedUniverse' / 'slcosmo').is_dir():
    repo_root = cwd / 'LensedUniverse'
else:
    for candidate in [cwd, *cwd.parents]:
        if (candidate / 'slcosmo').is_dir() and (candidate / 'data').is_dir():
            repo_root = candidate
            break

if repo_root is None:
    raise RuntimeError(f'Cannot locate LensedUniverse repo root from cwd={cwd}')

workdir = repo_root
os.chdir(workdir)
if str(workdir) not in sys.path:
    sys.path.insert(0, str(workdir))

from slcosmo.tools import tool

USE_X64 = os.environ.get('SLCOSMO_USE_X64', '0').strip().lower() in {'1', 'true', 'yes', 'y', 'on'}
jax.config.update('jax_enable_x64', USE_X64)
if USE_X64:
    numpyro.enable_x64()
if any(d.platform == 'gpu' for d in jax.devices()):
    numpyro.set_platform('gpu')
else:
    numpyro.set_platform('cpu')

print('Precision mode:', 'FP64' if USE_X64 else 'FP32')
print('Repo root:', workdir)
print('JAX devices:', jax.devices())

SEED = 42
rng_np = np.random.default_rng(SEED)
np.random.seed(SEED)

RESULT_DIR = workdir / 'test'
RESULT_DIR.mkdir(parents=True, exist_ok=True)


In [ ]:
# ---------------------------
# Load 8-lens dataset and keep only the longest delay pair per target
# ---------------------------
data_path = workdir / 'data' / 'test_data.py'
spec = importlib.util.spec_from_file_location('test_data', data_path)
test_data = importlib.util.module_from_spec(spec)
spec.loader.exec_module(test_data)
tdcosmo_8lens = test_data.tdcosmo_8lens

lens_names = sorted(tdcosmo_8lens.keys())

z_lens = []
z_src = []
sigma_v = []
sigma_v_err = []

pair_name = []
t_base = []
t_err = []

for name in lens_names:
    d = tdcosmo_8lens[name]
    z_lens.append(float(d['zl']))
    z_src.append(float(d['zs']))
    sigma_v.append(float(d['sigma_ap_los_kms']))
    sigma_v_err.append(float(d['sigma_ap_los_err_kms']))

    td_dict = d['time_delays_days']
    p_long = max(td_dict, key=lambda p: abs(float(td_dict[p]['dt'])))
    pair_name.append(f'{name}:{p_long}')
    dt_val = float(td_dict[p_long]['dt'])
    de = 0.5 * (float(td_dict[p_long].get('err_minus', 0.0)) + float(td_dict[p_long].get('err_plus', 0.0)))
    t_base.append(abs(dt_val))
    t_err.append(de)

z_lens = np.asarray(z_lens, dtype=float)
z_src = np.asarray(z_src, dtype=float)
sigma_v = np.asarray(sigma_v, dtype=float)
sigma_v_err = np.asarray(sigma_v_err, dtype=float)

t_base = np.asarray(t_base, dtype=float)
t_err = np.asarray(t_err, dtype=float)

n_lens = z_lens.size
n_obs = t_base.size

print('N lens:', n_lens)
print('N time-delay observations (longest pair only):', n_obs)
for i, name in enumerate(lens_names):
    print(f'  {i:02d} {name:15s} pair={pair_name[i]} sigma_v={sigma_v[i]:.1f}+-{sigma_v_err[i]:.1f}')



In [ ]:
# ---------------------------
# Build mock truth and observations
# ---------------------------
cosmo_true = {'Omegam': 0.3, 'Omegak': 0.0, 'w0': -1.0, 'wa': 0.0, 'h0': 70.0}

Dl, Ds, Dls = tool.dldsdls(jnp.asarray(z_lens), jnp.asarray(z_src), cosmo_true, n=20)
Ddt_lens = np.asarray((1.0 + z_lens) * Dl * Ds / Dls)
Ddt_obs = Ddt_lens

c_km_day = tool.c_km_s * 86400.0
Mpc_km = tool.Mpc / 1000.0

# Fixed MST population truth used to generate mock data
lambda_mean_true = 1.0
lambda_sigma_true = 0.1
lambda_low_model = 0.6
lambda_high_model = 1.6
lambda_true = tool.truncated_normal(
    lambda_mean_true,
    lambda_sigma_true,
    lambda_low_model,
    lambda_high_model,
    n_lens,
    random_state=rng_np,
)

# Per-lens external convergence truth and measurement error
# No population-level inference parameters for k_ext
kext_sigma = 0.01
kext_true = rng_np.normal(0.0, kext_sigma, n_lens)
kext_err = np.full(n_lens, 0.01)
kext_obs_clean = kext_true.copy()
kext_obs_noisy = rng_np.normal(kext_true, kext_err)

lambda_eff_true = (1.0 - kext_true) * lambda_true

# Individual MST measurement error from kinematics
lambda_err_frac_base = 2.0 * (sigma_v_err / sigma_v)
TDCOSMO_increase_error = 0.08
lambda_err_frac_plus8 = lambda_err_frac_base + TDCOSMO_increase_error

lambda_err_no_extra = lambda_err_frac_base * np.abs(lambda_true)
lambda_err_plus8 = lambda_err_frac_plus8 * np.abs(lambda_true)

# Time-delay measured true value: unbiased base delays transformed by lambda_eff truth
t_measured_true = t_base * lambda_eff_true

# Infer FPD true from t = (Ddt/c) * phi * lambda_eff
phi_true = (c_km_day * t_measured_true) / (Ddt_obs * Mpc_km * lambda_eff_true)
phi_err = 0.05 * np.abs(phi_true)

# Clean and noisy observation sets
# clean: observed values equal true values
# noisy: observed values sampled from measurement uncertainty
t_obs_clean = t_measured_true.copy()
phi_obs_clean = phi_true.copy()

t_obs_noisy = rng_np.normal(t_measured_true, t_err)
phi_obs_noisy = rng_np.normal(phi_true, phi_err)

# Scale phi for numerical stability
phi_scale = 10.0 ** int(np.round(-np.log10(np.median(np.abs(phi_true)))))
phi_obs_scaled_clean = phi_obs_clean * phi_scale
phi_obs_scaled_noisy = phi_obs_noisy * phi_scale
phi_err_scaled = 0.05 * np.abs(phi_true * phi_scale)

# Three scenarios
# - First two: infer lambda population (lambda_mean, lambda_sigma)
# - Third: lambda population is fixed and known
scenario_data = {
    'fiducial_no_extra': {
        'infer_lambda_population': True,
        'use_mst_measurement': True,
        'lambda_obs_clean': lambda_true.copy(),
        'lambda_obs_noisy': rng_np.normal(lambda_true, lambda_err_no_extra),
        'lambda_err': lambda_err_no_extra.copy(),
        'kext_obs_clean': kext_obs_clean.copy(),
        'kext_obs_noisy': kext_obs_noisy.copy(),
        'kext_err': kext_err.copy(),
    },
    'fiducial_plus8': {
        'infer_lambda_population': True,
        'use_mst_measurement': True,
        'lambda_obs_clean': lambda_true.copy(),
        'lambda_obs_noisy': rng_np.normal(lambda_true, lambda_err_plus8),
        'lambda_err': lambda_err_plus8.copy(),
        'kext_obs_clean': kext_obs_clean.copy(),
        'kext_obs_noisy': kext_obs_noisy.copy(),
        'kext_err': kext_err.copy(),
    },
    'population_only_no_mst_measure': {
        'infer_lambda_population': False,
        'use_mst_measurement': False,
        'lambda_obs_clean': np.zeros_like(lambda_true),
        'lambda_obs_noisy': np.zeros_like(lambda_true),
        'lambda_err': np.ones_like(lambda_true),
        'kext_obs_clean': kext_obs_clean.copy(),
        'kext_obs_noisy': kext_obs_noisy.copy(),
        'kext_err': kext_err.copy(),
    },
}

data_modes = {
    'clean': {
        't_obs': t_obs_clean,
        'phi_obs_scaled': phi_obs_scaled_clean,
    },
    'noisy': {
        't_obs': t_obs_noisy,
        'phi_obs_scaled': phi_obs_scaled_noisy,
    },
}

print('phi_scale:', phi_scale)
print('lambda_mean_true:', lambda_mean_true)
print('lambda_sigma_true:', lambda_sigma_true)
print('kext_sigma (true distribution):', kext_sigma)
print('kext_err (measurement):', np.unique(kext_err))
print('kext_true (%):', np.round(100.0 * kext_true, 3))
print('TDCOSMO_increase_error (%):', 100.0 * TDCOSMO_increase_error)
print('lambda_err_frac_base (%):', np.round(100.0 * lambda_err_frac_base, 2))
print('lambda_err_frac_plus8 (%):', np.round(100.0 * lambda_err_frac_plus8, 2))
print('scenario settings:', {k: (v['infer_lambda_population'], v['use_mst_measurement']) for k, v in scenario_data.items()})
print('data modes:', list(data_modes.keys()))



In [ ]:
# ---------------------------
# LambdaCDM model: sample Omegam/H0 and optionally infer lambda population
# ---------------------------
def quasar_td_lcdm_model(
    z_lens,
    z_src,
    t_obs,
    t_err,
    phi_obs_scaled,
    phi_err_scaled,
    phi_scale,
    lambda_obs,
    lambda_err,
    kext_obs,
    kext_err,
    use_mst_measurement,
    infer_lambda_population,
):
    Omegam = numpyro.sample('Omegam', dist.Uniform(0.1, 0.5))
    h0 = numpyro.sample('h0', dist.Uniform(0.0, 150.0))

    cosmo = {'Omegam': Omegam, 'Omegak': 0.0, 'w0': -1.0, 'wa': 0.0, 'h0': h0}

    z_lens = jnp.asarray(z_lens)
    z_src = jnp.asarray(z_src)
    t_obs = jnp.asarray(t_obs)
    t_err = jnp.asarray(t_err)
    phi_obs_scaled = jnp.asarray(phi_obs_scaled)
    phi_err_scaled = jnp.asarray(phi_err_scaled)
    phi_scale = jnp.asarray(phi_scale)
    lambda_obs = jnp.asarray(lambda_obs)
    lambda_err = jnp.asarray(lambda_err)
    kext_obs = jnp.asarray(kext_obs)
    kext_err = jnp.asarray(kext_err)

    if infer_lambda_population:
        lambda_mean = numpyro.sample(
            'lambda_mean',
            dist.Uniform(0.5, 1.5),
        )
        lambda_sigma = numpyro.sample(
            'lambda_sigma',
            dist.LogUniform(0.001, 0.5),
        )
    else:
        lambda_mean = jnp.asarray(lambda_mean_true)
        lambda_sigma = jnp.asarray(lambda_sigma_true)

    with numpyro.plate('lens', z_lens.shape[0]):
        lambda_lens = numpyro.sample(
            'lambda_true',
            dist.TruncatedNormal(lambda_mean, lambda_sigma, low=lambda_low_model, high=lambda_high_model),
        )
        kext_lens = numpyro.sample('kext', dist.Normal(0.0, kext_sigma))
        if use_mst_measurement:
            numpyro.sample('lambda_like', dist.Normal(lambda_lens, lambda_err), obs=lambda_obs)
        numpyro.sample('kext_like', dist.Normal(kext_lens, kext_err), obs=kext_obs)

    lambda_eff = (1.0 - kext_lens) * lambda_lens

    Dl, Ds, Dls = tool.dldsdls(z_lens, z_src, cosmo, n=20)
    Ddt_lens = (1.0 + z_lens) * Dl * Ds / Dls
    Ddt_obs = Ddt_lens

    c_km_day = tool.c_km_s * 86400.0
    Mpc_km = tool.Mpc / 1000.0

    with numpyro.plate('td_obs', t_obs.shape[0]):
        phi_latent_scaled = numpyro.sample('phi_true_scaled', dist.Normal(phi_obs_scaled, phi_err_scaled))
        phi_latent = phi_latent_scaled / phi_scale
        t_model_days = (Ddt_obs * Mpc_km / c_km_day) * phi_latent * lambda_eff
        numpyro.sample('t_delay_like', dist.Normal(t_model_days, t_err), obs=t_obs)


def build_init_values(phi_obs_scaled, infer_lambda_population):
    values = {
        'Omegam': jnp.asarray(cosmo_true['Omegam']),
        'h0': jnp.asarray(cosmo_true['h0']),
        'lambda_true': jnp.asarray(lambda_true),
        'kext': jnp.asarray(kext_true),
        'phi_true_scaled': jnp.asarray(phi_obs_scaled),
    }
    if infer_lambda_population:
        values['lambda_mean'] = jnp.asarray(lambda_mean_true)
        values['lambda_sigma'] = jnp.asarray(lambda_sigma_true)
    return values



In [ ]:
# ---------------------------
# Run three scenarios with clean and noisy data
# ---------------------------
RUN_MCMC = True
TARGET_ACCEPT = 0.95
NUM_WARMUP = 500
NUM_SAMPLES = 1000
NUM_CHAINS = 4


def run_mcmc(scenario_name, scenario, mode_name, mode_data, key):
    init_values = build_init_values(mode_data['phi_obs_scaled'], bool(scenario['infer_lambda_population']))
    nuts = NUTS(
        quasar_td_lcdm_model,
        target_accept_prob=TARGET_ACCEPT,
        init_strategy=init_to_value(values=init_values),
    )
    mcmc = MCMC(
        nuts,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        chain_method='vectorized',
        progress_bar=True,
    )

    mcmc.run(
        key,
        z_lens=z_lens,
        z_src=z_src,
        t_obs=mode_data['t_obs'],
        t_err=t_err,
        phi_obs_scaled=mode_data['phi_obs_scaled'],
        phi_err_scaled=phi_err_scaled,
        phi_scale=phi_scale,
        lambda_obs=scenario[f'lambda_obs_{mode_name}'],
        lambda_err=scenario['lambda_err'],
        kext_obs=scenario[f'kext_obs_{mode_name}'],
        kext_err=scenario['kext_err'],
        use_mst_measurement=bool(scenario['use_mst_measurement']),
        infer_lambda_population=bool(scenario['infer_lambda_population']),
    )

    extra = mcmc.get_extra_fields(group_by_chain=True)
    n_div = int(np.asarray(extra['diverging']).sum())
    print(f'[{scenario_name} | {mode_name}] divergences:', n_div)

    posterior = mcmc.get_samples(group_by_chain=True)
    inf_data = az.from_dict(posterior=posterior)
    return inf_data


def make_single_corner(idata, var_names, outfile, color):
    fig = corner(
        idata,
        var_names=var_names,
        labels=var_names,
        color=color,
        show_titles=False,
        levels=[0.68, 0.95],
        fill_contours=True,
        plot_datapoints=False,
        smooth=0.2,
        use_math_text=True,
        contour_kwargs={'linewidths': 2.5},
        hist_kwargs={'density': True, 'linewidth': 2.5},
    )
    fig.savefig(outfile, bbox_inches='tight')
    plt.show()
    plt.close(fig)


if RUN_MCMC:
    order = ['fiducial_no_extra', 'fiducial_plus8', 'population_only_no_mst_measure']
    mode_order = ['clean', 'noisy']

    keys = random.split(random.PRNGKey(SEED), len(order) * len(mode_order))
    key_idx = 0

    inference_results = {mode: {} for mode in mode_order}

    for mode in mode_order:
        for i, name in enumerate(order):
            scenario = scenario_data[name]
            mode_data = data_modes[mode]
            inf_data = run_mcmc(name, scenario, mode, mode_data, keys[key_idx])
            key_idx += 1
            inference_results[mode][name] = inf_data

            nc_path = RESULT_DIR / f'test_quasar8_{name}_{mode}.nc'
            az.to_netcdf(inf_data, nc_path)

            trace_vars = ['h0', 'Omegam']
            if scenario['infer_lambda_population']:
                trace_vars += ['lambda_mean', 'lambda_sigma']

            axes = az.plot_trace(inf_data, var_names=trace_vars, compact=True)
            fig_trace = axes.ravel()[0].figure
            trace_path = RESULT_DIR / f'test_quasar8_{name}_{mode}_trace.pdf'
            fig_trace.savefig(trace_path, bbox_inches='tight')
            plt.close(fig_trace)

            print(f"\n[{name} | {mode}] arviz summary")
            print(az.summary(inf_data, var_names=trace_vars, round_to=4))

            corner_path = RESULT_DIR / f'test_quasar8_{name}_{mode}_corner.pdf'
            make_single_corner(
                inf_data,
                trace_vars,
                corner_path,
                '#2f8aed' if i == 0 else ('#f48c06' if i == 1 else '#2ca25f'),
            )

            print('Saved:', nc_path)
            print('Saved:', trace_path)
            print('Saved:', corner_path)

        # Overlay across scenarios for each data mode (common cosmology only)
        fig = corner(
            inference_results[mode][order[0]],
            var_names=['h0', 'Omegam'],
            labels=['h0', 'Omegam'],
            color='#2f8aed',
            show_titles=False,
            levels=[0.68, 0.95],
            fill_contours=True,
            plot_datapoints=False,
            smooth=0.2,
            use_math_text=True,
            contour_kwargs={'linewidths': 2.5},
            hist_kwargs={'density': True, 'linewidth': 2.5},
        )
        corner(
            inference_results[mode][order[1]],
            fig=fig,
            var_names=['h0', 'Omegam'],
            labels=['h0', 'Omegam'],
            color='#f48c06',
            show_titles=False,
            levels=[0.68, 0.95],
            fill_contours=True,
            plot_datapoints=False,
            smooth=0.2,
            use_math_text=True,
            contour_kwargs={'linewidths': 2.5},
            hist_kwargs={'density': True, 'linewidth': 2.5},
        )
        corner(
            inference_results[mode][order[2]],
            fig=fig,
            var_names=['h0', 'Omegam'],
            labels=['h0', 'Omegam'],
            color='#2ca25f',
            show_titles=False,
            levels=[0.68, 0.95],
            fill_contours=True,
            plot_datapoints=False,
            smooth=0.2,
            use_math_text=True,
            contour_kwargs={'linewidths': 2.5},
            hist_kwargs={'density': True, 'linewidth': 2.5},
        )
        overlay_path = RESULT_DIR / f'test_quasar8_overlay_h0_Omegam_{mode}.pdf'
        fig.savefig(overlay_path, bbox_inches='tight')
        plt.show()
        plt.close(fig)
        print('Saved:', overlay_path)


